# Movie Recommendation System -  top 5 Movies
 - Content Based Recommender System

# Import - Important libraries
 - Pandas for cleaning and Exploring purpose 
 - Numpy for mathematical conditions 
 - Sklearn for vectorization 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Load Movie Data
 - Movies 
 - Credits

In [ ]:
movies = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")
credits = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv")

In [ ]:
movies.head(3)

In [ ]:
credits.head(2)

# Joining two dataframs
 - Movies and Credits datafram on the basis of title

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.head(2)

In [ ]:
movies.info()

# Recoganizing Importent columns Importent columns 
 - Genres
 - Keywords
 - title
 - Overview 
 - id
 - cast
 - crew


In [ ]:
movies = movies[['id','genres','keywords','title','overview','cast','crew']]

In [ ]:
movies.head(1)

Checking missing data

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

# convert Object list into list

In [ ]:
import ast

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head(2)

In [ ]:
movies['keywords'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
## movies['cast'][0]

In [ ]:
#For Cast name 
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+= 1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head(3)

In [ ]:
## movies['crew'][0]

In [ ]:
def fetchdirector(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetchdirector)

In [ ]:
movies.head(3)

In [ ]:
movies.overview[0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.dtypes

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

# creating tags by concatination following columns -  
 - overview
 - genres
 - keywords 
 - casta 
 - crew

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies ['crew']

In [ ]:
movies.head(5)

In [ ]:
new_df = movies[['id','title','tags']]

In [ ]:
#list into string conversion 
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
#convert it into lower case for best practise 
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors[1]

In [ ]:
cv.get_feature_names()

In [ ]:
#stemming ['loved','loving'] to ['love','love']

In [ ]:
#ecledian distance. -  for higher data set fails 
#Use -  cosine Distance 
# distance x 1/similarity 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movies_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movies_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

## Function calling for recommendation 

In [ ]:
recommend('Gandhi, My Father')

In [ ]:
import pickle

In [ ]:
new_df.sample(10)